In [1]:
import numpy as np

# import stuff for creating class

from abc import ABC, abstractmethod


In [2]:
from sklearn.preprocessing import normalize
class TfIdfBase(ABC):
    def __init__(self, corpus, docids, save_folder):
        self.corpus = corpus
        self.docids = docids
        self.save_folder = save_folder

    @abstractmethod
    def compute_idf(self):
        pass

    @abstractmethod
    def compute_tf(self):
        pass

    @abstractmethod
    def preprocess(self):
        pass

    def compute_tf_idf(self, do_normalize=True):
        self.tf_idf_matrix = self.tf_matrix * self.idf_matrix
        # row-wise L2 norm
        if do_normalize:
            self.tf_idf_matrix = normalize(self.tf_idf_matrix)

In [3]:
import stopwordsiso as stopwords
import re
from loguru import logger
from tqdm.auto import tqdm
import time
import pickle
import scipy.sparse
import os
import collections

class TfIdfGerman(TfIdfBase):
    def __init__(self, corpus, docids, save_folder, overwrite=False):
        super().__init__(corpus, docids, save_folder)
        self.stopwords = stopwords.stopwords("de")
        self.overwrite = overwrite

        os.makedirs(f'{self.save_folder}/de', exist_ok=True)

    def preprocess(self):

        if not self.overwrite and os.path.exists(f'{self.save_folder}/de/word_list.pkl'):
            logger.info("Loading unique words and word list")
            with open(f'{self.save_folder}/de/word_list.pkl', 'rb') as f:
                self.unique_words, self.word_list, self.word_to_index = pickle.load(f)

            return  

        logger.info("Preprocessing corpus")
        start = time.perf_counter()
        # self.corpus = [re.sub(r'[^\w\s]', doc.lower()) for doc in tqdm(self.corpus)]
        # self.corpus = [[word for word in doc.split() if word not in self.stopwords] for doc in tqdm(self.corpus)]

        # combine the above 2 into 1
        self.word_list = [[word for word in re.sub(r'[^\w\s]', '', doc.lower()).split()\
                        if word not in self.stopwords] for doc in tqdm(self.corpus)]


        end = time.perf_counter()

        self.unique_words = set()
        for doc in tqdm(self.word_list):
            self.unique_words.update(doc)

        self.unique_words = sorted(list(self.unique_words))
        self.word_to_index = {word: i for i, word in enumerate(self.unique_words)}

        logger.info(f"Preprocessing took {end-start:.2f} seconds")
        logger.info(f"Number of unique words: {len(self.unique_words)}")

        logger.info(f"Saving unique words and word list to {self.save_folder}/de")
        with open(f'{self.save_folder}/de/word_list.pkl', 'wb') as f:
            pickle.dump((self.unique_words, self.word_list, self.word_to_index), f)

    def compute_tf(self):
        # computing term frequency

        # if not self.overwrite and os.path.exists(f'{self.save_folder}/en/tf_matrix.npy'):
        #     logger.info("Loading tf matrix")
        #     self.tf_matrix = np.load(f'{self.save_folder}/en/tf_matrix.npy')
        #     return self.tf_matrix

        # logger.info("Computing term frequency")
        # start = time.perf_counter()

        # self.tf_matrix = scipy.sparse.csr_matrix(np.zeros((len(self.docids), len(self.unique_words)), dtype=np.int32))

        # for i, doc in enumerate(tqdm(self.word_list)):
        #     for word in doc:
        #         self.tf_matrix[i, self.word_to_index[word]] += 1

        # end = time.perf_counter()
        # logger.info(f"Computing term frequency took {end-start:.2f} seconds")

        # logger.info(f"Saving tf matrix to {self.save_folder}/en")
        # np.save(f'{self.save_folder}/en/tf_matrix.npy', self.tf_matrix)
        # return self.tf_matrix
        self._compute_sparse_tf()
    

    def _compute_sparse_tf(self):
        if not self.overwrite and os.path.exists(f'{self.save_folder}/de/sparse_tf_matrix.npy'):
            logger.info("Loading tf matrix")
            self.tf_matrix = np.load(f'{self.save_folder}/de/sparse_tf_matrix.npy', allow_pickle=True)
            return self.tf_matrix
        start = time.perf_counter()
        row, col, data = [], [], []
        for i, doc in enumerate(tqdm(self.word_list)):
            word_count = collections.defaultdict(int)
            for word in doc:
                word_count[word] += 1


            for word, count in word_count.items():
                row.append(i)
                col.append(self.word_to_index[word])
                data.append(count)

        self.tf_matrix = scipy.sparse.csr_matrix((data, (row, col)), shape=(len(self.docids), len(self.unique_words)), dtype=np.int32)
        end = time.perf_counter()
        logger.info(f"Computing term frequency took {end-start:.2f} seconds")

        logger.info(f"Saving tf matrix to {self.save_folder}/de")
        np.save(f'{self.save_folder}/de/sparse_tf_matrix.npy', self.tf_matrix)
        # return self.tf_matrix


    def _compute_sparse_idf(self):
        if not self.overwrite and os.path.exists(f'{self.save_folder}/de/sparse_idf_matrix.npy'):
            logger.info("Loading idf matrix")
            self.idf_matrix = np.load(f'{self.save_folder}/de/sparse_idf_matrix.npy', allow_pickle=True)
            return self.idf_matrix

        start = time.perf_counter()
        self.idf_matrix = (self.tf_matrix > 0).sum(axis=0)
        self.idf_matrix = np.squeeze(np.array(self.idf_matrix))
        self.idf_matrix = np.log((1 + len(self.docids)) / (1 + self.idf_matrix)) + 1
        print(self.idf_matrix)
        self.idf_matrix = scipy.sparse.diags(self.idf_matrix.astype(np.float32))
        end = time.perf_counter()
        logger.info(f"Computing inverse document frequency took {end-start:.2f} seconds")

        logger.info(f"Saving idf matrix to {self.save_folder}/de")
        np.save(f'{self.save_folder}/de/sparse_idf_matrix.npy', self.idf_matrix)
        # return self.idf_matrix
    
    def compute_idf(self):
        # return self._compute_sparse_idf()
        self._compute_sparse_idf()


    def _get_sparse_query_tfidf(self, queries, use_idf, do_normalize):
        query_word_list = [[word for word in re.sub(r'[^\w\s]', '', query.lower()).split()\
                        if word not in self.stopwords] for query in queries]

        # query_vectors = np.zeros((len(queries), len(self.unique_words)))
        # for i in range(len(queries)):
        #     for word in query_word_list[i]:
        #         if word in self.word_to_index:
        #             query_vectors[i][self.word_to_index[word]] = 

        row, col, data = [], [], []

        for i, doc in enumerate(query_word_list):
            word_count = collections.defaultdict(int)
            for word in doc:
                if word in self.word_to_index:
                    word_count[word] += 1

            for word, count in word_count.items():
                row.append(i)
                col.append(self.word_to_index[word])
                data.append(count)

        query_tf_idf_matrix = scipy.sparse.csr_matrix((data, (row, col)), shape=(len(queries), len(self.unique_words)), dtype=np.int32)

        if use_idf:
            query_tf_idf_matrix = np.array(query_tf_idf_matrix)
            query_tf_idf_matrix = query_tf_idf_matrix * self.idf_matrix
        
        if do_normalize:
            query_tf_idf_matrix = normalize(query_tf_idf_matrix)

        return query_tf_idf_matrix



    def process_queries(self, queries, topk=5, use_idf=False, do_normalize=True):

        assert len(queries) == 1
        query_tf_idf_matrix = self._get_sparse_query_tfidf(queries, use_idf, do_normalize)

        cosine_similarities = self.tf_idf_matrix.dot(query_tf_idf_matrix.T).toarray().flatten()
        top_k_indices = np.argsort(cosine_similarities)[::-1][:topk]  # Get top 10 indices by score

        return [self.docids[i] for i in top_k_indices]



In [7]:
# english_corpus = [
#     "this is the first document psosp",
#     "this document is the second document ssdfa",
#     "and this is the third one",
#     "is this the first document"
# ]
# english_docids = list(range(len(english_corpus)))


In [4]:
import json

with open('corpus.json') as f:
    corpus = json.load(f)

In [5]:
german_corpus = [doc['text'] for doc in corpus if doc['lang'] == 'de']
german_docids = [doc['docid'] for doc in corpus if doc['lang'] == 'de']

In [6]:
TFV = TfIdfGerman(german_corpus, german_docids, 'tfidf', overwrite=False)

In [7]:
TFV.preprocess()

2024-10-18 16:17:49.712 | INFO     | __main__:preprocess:28 - Preprocessing corpus


  0%|          | 0/10992 [00:00<?, ?it/s]

  0%|          | 0/10992 [00:00<?, ?it/s]

2024-10-18 16:18:26.064 | INFO     | __main__:preprocess:47 - Preprocessing took 16.42 seconds
2024-10-18 16:18:26.069 | INFO     | __main__:preprocess:48 - Number of unique words: 2454116
2024-10-18 16:18:26.070 | INFO     | __main__:preprocess:50 - Saving unique words and word list to tfidf/de


In [8]:
TFV.compute_tf()

  0%|          | 0/10992 [00:00<?, ?it/s]

2024-10-18 16:19:09.490 | INFO     | __main__:_compute_sparse_tf:100 - Computing term frequency took 18.02 seconds
2024-10-18 16:19:09.492 | INFO     | __main__:_compute_sparse_tf:102 - Saving tf matrix to tfidf/de


In [9]:
TFV.compute_idf()

2024-10-18 16:19:15.625 | INFO     | __main__:_compute_sparse_idf:120 - Computing inverse document frequency took 0.19 seconds
2024-10-18 16:19:15.625 | INFO     | __main__:_compute_sparse_idf:122 - Saving idf matrix to tfidf/de


[2.56914367 3.89155503 3.48172786 ... 9.61186681 9.61186681 9.61186681]


In [10]:
TFV.compute_tf_idf()

In [11]:
import pandas as pd
dev_set = pd.read_csv('./dev.csv')
german_dev_set = dev_set[dev_set['lang'] == 'de']
german_dev_set.shape

(200, 5)

In [12]:
german_dev_set_queries = german_dev_set['query'].tolist()
german_dev_set_positive_docs = german_dev_set['positive_docs'].tolist()

In [13]:
def recall_at_10(positive_docs, top_10_ids):
    recall = []
    for positive_doc, top_10_id in zip(positive_docs, top_10_ids):
        recall.append(positive_doc in top_10_id)

    print(np.mean(recall))

In [14]:
top_10_ids = []
for dev_query in tqdm(german_dev_set_queries):
    top_10_ids.append(TFV.process_queries([dev_query], 10, True, True))

  0%|          | 0/200 [00:00<?, ?it/s]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [19]:
recall_at_10(german_dev_set_positive_docs, top_10_ids)

0.63


In [20]:
top_10_ids = []
for dev_query in tqdm(german_dev_set_queries):
    top_10_ids.append(TFV.process_queries([dev_query], 10, False, True))

    

  0%|          | 0/200 [00:00<?, ?it/s]

In [21]:
top_10_ids = []
for dev_query in tqdm(german_dev_set_queries):
    top_10_ids.append(TFV.process_queries([dev_query], 10, True, False))

    

  0%|          | 0/200 [00:00<?, ?it/s]

In [22]:
recall_at_10(german_dev_set_positive_docs, top_10_ids)

0.63


In [23]:
# svd trial
from sklearn.decomposition import TruncatedSVD

In [24]:
svd = TruncatedSVD(n_components=512)

In [ ]:
reduced_tfidf_matrix = svd.fit_transform(TFV.tf_idf_matrix)